In [1]:
import numpy as np
from numba import jit
import time
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import math
from scipy.stats import norm

## Part II: Estimation of Sensitivities in MC

In [24]:
def Delta_Analytical_Call(S, K, r, sigma, tau):
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * tau) / (sigma * math.sqrt(tau))
    return norm.cdf(d1)

def Delta_Analytical_Put(S, K, r, sigma, tau):
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * tau) / (sigma * math.sqrt(tau))
    return norm.cdf(d1) - math.e**-r

In [25]:
@jit(nopython=True, fastmath=True, parallel=True)
def European_Put(T, K, r, S, sigma, trials):
    '''
    This function calculates the value of an European Put option
    Arguments: maturity, strike price, interest rate, stock price, volaility, number of trials
    Returns: Array of size trial with values of european puts
    '''
    
    S_adjust = S * np.exp(r - (0.5 * sigma**2)*T)
    payoff_array = np.zeros(trials)
    
    for i in range(trials):
        S_cur = S_adjust * np.exp(sigma*np.sqrt(T)*np.random.normal())
        
        if K-S_cur > 0:
            payoff_array[i] = (K-S_cur)*np.exp(-r*T)
        else:
            payoff_array[i] = 0

    return payoff_array


@jit(nopython=True, fastmath=True, parallel=True)
def European_Call(T, K, r, S, sigma, trials):
    '''
    This function calculates the value of an European Put option
    Arguments: maturity, strike price, interest rate, stock price, volaility, number of trials
    Returns: Array of size trial with values of european puts
    '''
    
    S_adjust = S * np.exp(r - (0.5 * sigma**2)*T)
    payoff_array = np.zeros(trials)
    
    for i in range(trials):
        S_cur = S_adjust * np.exp(sigma*np.sqrt(T)*np.random.normal())
        
        if S_cur-K > 0:
            payoff_array[i] = (S_cur-K)*np.exp(-r*T)
        else:
            payoff_array[i] = 0

    return payoff_array


def get_delta_put(kwargs, S, e, seed=None):
    if seed:
        np.random.seed(seed)
    kwargs['S'] = S
    V = European_Put(**kwargs)
    
    
    kwargs['S'] = S + e
    if seed:
        np.random.seed(seed)
    Ve = European_Put(**kwargs)
    
    return (Ve-V)/ e


def get_delta_call(kwargs, S, e, seed=None):
    if seed:
        np.random.seed(seed)
        
    kwargs['S'] = S
    V = European_Call(**kwargs)
    
    
    kwargs['S'] = S + e
    if seed:
        np.random.seed(seed)
    Ve = European_Call(**kwargs)
    
    return (Ve-V)/ e

In [26]:
Delta_Analytical_Call(100, 99, 0.06, 0.2, 1)

0.6737355117348961

In [27]:
kwargs = {}
kwargs['T'] = 1
kwargs['K'] = 99
kwargs['r'] = 0.06
kwargs['sigma'] = 0.2
kwargs['trials'] = 10**6

In [28]:
S = 100
e = 1
delta_matrix = [get_delta_call(kwargs, S, e, 40) for x in range(100)]

In [29]:
np.mean(delta_matrix)

0.6838943664261715

In [30]:
np.std(delta_matrix) / np.sqrt(len(delta_matrix[0]) * len(delta_matrix))

0.0021944289134009955

In [18]:
kwargs['trials'] = 100_000_000
kwargs['S'] = 100
np.mean(European_Put(**kwargs))

4.7787637986912275

In [ ]:
kwargs['trials'] = 100_000_000
kwargs['S'] = 105
np.mean(European_Put(**kwargs))